In [1]:
def plot_time_series(df_load, df_weather, bldg_id):
    """
    Function to create a time series plot with energy consumption, temperature, and humidity.

    Parameters:
    - df_load (pd.DataFrame): DataFrame containing load data with 'timestamp' column.
    - df_weather (pd.DataFrame): DataFrame containing weather data with 'date_time' column.
    - bldg_id (str): Building ID for the plot title.
    """
    # Convert 'timestamp' and 'date_time' columns to datetime
    df_load['timestamp'] = pd.to_datetime(df_load['timestamp'])
    df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])

    # Rename columns for merging
    df_load.rename(columns={'timestamp': 'date_time'}, inplace=True)

    # Merge dataframes on 'date_time'
    df_merged = pd.merge(df_load, df_weather, on='date_time', how='inner')

    # Melt the dataframe to long format for Plotly Express
    df_long = df_merged.melt(id_vars='date_time',
                             value_vars=['out.electricity.total.energy_consumption',
                                         'Dry Bulb Temperature [°C]',
                                         'Relative Humidity [%]'],
                             var_name='Measurement',
                             value_name='Value')

    # Replace specific measurement names for legend
    df_long['Measurement'] = df_long['Measurement'].replace(
        'out.electricity.total.energy_consumption', 'Energy Consumption')

    # Create time series plot with Plotly Express
    fig = px.line(df_long, x='date_time', y='Value', color='Measurement',
                  labels={'Value': 'Measurement', 'date_time': 'Time'},
                  title=f"Building: {bldg_id}")

    # Update layout for better y-axis management
    fig.update_layout(
        yaxis_title='Measurement',
        yaxis=dict(title='Measurement'),
        yaxis2=dict(
            title='Temperature [°C]',
            overlaying='y',
            side='right',
            anchor='x',
            position=0.85
        ),
        yaxis3=dict(
            title='Humidity [%]',
            overlaying='y',
            side='right',
            anchor='x',
            position=1
        )
    )

    # Show plot
    fig.show()